In [8]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [20]:
!pip install -q Cython torch

In [9]:
pip install spacy

In [12]:
!pip install -U spacy

In [15]:
!pip install Cython


In [2]:
pip install pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.1 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140085 sha256=5f8c089ecf7d725b45322e0d2d7ef466bac14b4bd1abed5be4b014a4040de4a2
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer


In [1]:
pip install pdfminer.six

In [2]:
from pdfminer.high_level import extract_text

In [16]:
import re
import spacy
from pdfminer.high_level import extract_text
import pandas as pd

# Load English tokenizer, POS tagger, parser, NER, and word vectors
nlp = spacy.load('en_core_web_sm')

# Function to extract text from a PDF resume
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

# Function to extract email address
def extract_email(text):
    email = re.findall(r'\S+@\S+', text)
    return email[0] if email else None

# Function to extract phone number
def extract_phone_number(text):
    phone = re.findall(r'\(?\b[0-9]{3}\)?[-.\s]?[0-9]{3}[-.\s]?[0-9]{4}\b', text)
    return phone[0] if phone else None

# Function to extract LinkedIn profile
def extract_linkedin(text):
    linkedin = re.findall(r'https?://(?:www\.)?linkedin\.com/in/[a-zA-Z0-9_-]+', text)
    return linkedin[0] if linkedin else None

# Function to extract GitHub profile
def extract_github(text):
    github = re.findall(r'https?://(?:www\.)?github\.com/[a-zA-Z0-9_-]+', text)
    return github[0] if github else None

# Function to extract GPA or percentage
def extract_gpa(text):
    gpa = re.findall(r'\bGPA\b[\s:]*([0-9]\.\d{1,2}|\d{1,2}\.\d{1,2})', text)
    if not gpa:
        percentage = re.findall(r'\b(?:percentage|percent)\b[\s:]*([0-9]{1,2}\.\d{1,2})\%', text, re.I)
        return percentage[0] if percentage else None
    return gpa[0]

# Function to extract name (basic heuristic approach)
def extract_name(nlp_text, matcher):
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', [pattern])

    matches = matcher(nlp_text)
    for _, start, end in matches:
        span = nlp_text[start:end]
        return span.text

# Function to extract skills
def extract_skills(nlp_text):
    skills = ["c","c++","python", "java", "sql", "machine learning", "data analysis", "excel", "tableau", "nlp", "deep learning", "pandas", "numpy", "scikit-learn", "tensorflow"]
    skillset = set()  # Use a set to avoid duplicates
    for token in nlp_text:
        if token.text.lower() in skills:
            skillset.add(token.text.lower())
    return list(skillset)

# Function to extract education qualifications with status
def extract_education(text):
    degree_keywords = ['bachelor', 'master of technology', 'bsc', 'msc', 'phd', 'b.tech', 'm.tech', 'bachelor of science', 'master of science', 'engineering', 'computer science', 'information technology']
    status_keywords = ['pursuing', 'currently', 'ongoing', 'expected', 'completed']

    sentences = text.lower().split('\n')
    education = []

    for sentence in sentences:
        degree_info = None
        status_info = None

        # Check for degree keywords
        for keyword in degree_keywords:
            if keyword in sentence:
                degree_info = keyword
                break

        # Check for status keywords
        for status in status_keywords:
            if status in sentence:
                status_info = status
                break

        if degree_info:
            if status_info:
                # Format the degree information with status
                formatted_education = f"{degree_info.title()} ({status_info})"
            else:
                # No status information
                formatted_education = degree_info.title()
            education.append(formatted_education)

    return education

# Main function to parse resume and display in a tabular format
def parse_resume(pdf_path):
    # Extract text from the PDF
    text = extract_text_from_pdf(pdf_path)

    # Process text with spaCy
    nlp_text = nlp(text)

    # Initialize the matcher
    matcher = spacy.matcher.Matcher(nlp.vocab)

    # Extract details
    name = extract_name(nlp_text, matcher)
    email = extract_email(text)
    phone = extract_phone_number(text)
    linkedin = extract_linkedin(text)
    github = extract_github(text)
    gpa = extract_gpa(text)
    skills = ', '.join(extract_skills(nlp_text))
    education = ', '.join(extract_education(text))

    # Create a DataFrame to display the result in a tabular format
    data = {
        'Field': ['Name', 'Email', 'Phone', 'LinkedIn', 'GitHub', 'GPA/Percentage', 'Skills', 'Education'],
        'Value': [name, email, phone, linkedin, github, gpa, skills, education]
    }
    df = pd.DataFrame(data)

    # Display the DataFrame
    return df

# Example usage
pdf_path = '/content/Om_Resume.pdf'  # Replace with your resume file path
parse_resume(pdf_path)


,Field,Value
0,Name,Adavala Om
1,Email,omadavala8@gmail.com
2,Phone,8500004279
3,LinkedIn,None
4,GitHub,None
5,GPA/Percentage,None
6,Skills,"java, python, c++, sql, c"
7,Education,"Master Of Technology, Master Of Technology (pu..."
